In [2]:
import cv2
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np


model =Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

TRAINING_DIR = "Dataset/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, batch_size=10, 
target_size=(150, 150))

VALIDATION_DIR = "Dataset/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, batch_size=10, target_size=(150, 150))

history = model.fit_generator(train_generator, epochs=10, validation_data=validation_generator)
model.save('facemaskmodel.h5')

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.
Epoch 1/10
132/132 [==============================] - 415s 3s/step - loss: 0.6554 - acc: 0.6730 - val_loss: 0.1551 - val_acc: 0.9588
Epoch 2/10
132/132 [==============================] - 373s 3s/step - loss: 0.2902 - acc: 0.8923 - val_loss: 0.2561 - val_acc: 0.9278
Epoch 3/10
132/132 [==============================] - 401s 3s/step - loss: 0.2988 - acc: 0.9004 - val_loss: 0.0996 - val_acc: 0.9639
Epoch 4/10
132/132 [==============================] - 367s 3s/step - loss: 0.2747 - acc: 0.8937 - val_loss: 0.1937 - val_acc: 0.9124
Epoch 5/10
132/132 [==============================] - 363s 3s/step - loss: 0.3292 - acc: 0.8784 - val_loss: 0.0590 - val_acc: 0.9845
Epoch 6/10
132/132 [==============================] - 365s 3s/step - loss: 0.1977 - acc: 0.9368 - val_loss: 0.0730 - val_acc: 0.9845
Epoch 7/10
132/132 [==============================] - 376s 3s/step - loss: 0.1799 - acc: 0.9391 - val_loss: 0.2688 - va

In [5]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("facemaskmodel.h5")
labels_dict={0:'without mask',1:'mask'}
color_dict={0:(0,0,255),1:(0,255,0)}
size = 4
webcam = cv2.VideoCapture(0) #Use camera 0
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))
    faces = classifier.detectMultiScale(mini)
    for f in faces:
        (x, y, w, h) = [v * size for v in f]
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    if key == 27:
        break
webcam.release()
cv2.destroyAllWindows()